In [ ]:
# =============================================================================
# Authors: Deepthi Kuttichira and Pubudu Sanjeewani Thihagoda Gamage
# Date: 20-08-2024
# Title: A Novel Non-Iterative Training Method for CNN Classifiers Using Gram-Schmidt Process
# License: MIT License
# =============================================================================

# Import necessary libraries
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from sklearn.metrics import precision_score

# Load MNIST dataset and extract labels for later evaluation
(x_train,y_train),(x_test,y_test)=mnist.load_data()
y_test_check=y_test
y_train_check=y_train

# Load precomputed feature vectors for training and testing (Change the file path to the actual location on the system)
# Remove columns with all-zero values to reduce redundancy
x_train=np.load(r'C:\Users\sanje\Desktop\Griffith\code\pretrained-MNIST\mnisttrain_2000relu2.npy')
l=np.sum(x_train,axis=0) 
index=np.where(l==0)[0]
x_train=np.delete(x_train,index,axis=1)
x_test=np.load(r'C:\Users\sanje\Desktop\Griffith\code\pretrained-MNIST\mnisttest_2000relu2.npy')
x_test = np.delete(x_test,index,axis=1)
print(x_test.shape)

# Load precomputed QR weights
weights=np.load("qrweightmnist_2000relu2.npy")

# One-hot encode the labels for training/testing
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

# Define and compile a simple classifier model
optimizer='adam'
input_dim = 946 # Update this if the input feature dimension changes e.g., 958 for VGG19, 256 for ResNet50, 941 for DeneseNet121, and  934 for DenseNet169
output_dim = 10 

def classification_model():
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Initialize model and set pretrained weights and biases
keras_model = classification_model()

l=[]
x=weights.transpose() # Weight matrix (input_dim x output_dim)
y=np.array([0]*10)    # Bias vector (set to zeros)
l.append(x)
l.append(y)
keras_model.layers[0].set_weights(l)

# Predict class labels and compute precision scores
prediction_class_test = keras_model.predict(x_test).argmax(axis=-1)
accuracy_test = precision_score(prediction_class_test, y_test_check, average='micro')

prediction_class_train = keras_model.predict(x_train).argmax(axis=-1)
accuracy_train = precision_score(prediction_class_train, y_train_check, average='micro')

# Evaluate and print training and testing loss and accuracy
train_loss, train_accuracy = keras_model.evaluate(x_train, y_train, verbose=0)
test_loss, test_accuracy = keras_model.evaluate(x_test, y_test, verbose=0)

print(f'Train Loss: {train_loss}, Train Accuracy: {train_accuracy}')
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

(10000, 946)
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Train Loss: 1.4645800590515137, Train Accuracy: 0.9979666471481323
Test Loss: 1.4684962034225464, Test Accuracy: 0.9940999746322632
